<a href="https://colab.research.google.com/github/jefernandezec/s2s/blob/main/S2S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamentals of Survey-to-Survey Imputation

**0. Setup**

In [ ]:
install.packages("glmnet")
library(glmnet)
library(ggplot2)


**1. Creating a dataset with a model-based consumption per capita**

Set seed for reproducibility and simulate raw variables

In [ ]:
set.seed(1729)
n <- 1000

age             <- sample(18:79, n, replace = TRUE)
age_sq          <- age^2
house_ownership <- rbinom(n, 1, 0.6)   # 1 = owns house
employed    <- rbinom(n, 1, 0.7)   # 1 = employed
electricity     <- rbinom(n, 1, 0.8)   # 1 = has electricity
wallsbrick <- rbinom(n, 1, 0.75)   # 1 = walls material: brick
roofmetal  <- rbinom(n, 1, 0.25)   # 1 = roof material: brick
floordirt  <- rbinom(n, 1, 0.15)   # 1 = floor material: dirt
areaurban  <- rbinom(n, 1, 0.65)   # 1 = urban
hhsize     <- sample(1:8, n, replace = TRUE)

df <- data.frame(
  age, age_sq, house_ownership, employed, electricity,
  wallsbrick,roofmetal,floordirt,areaurban,hhsize
)

Define "actual" coefficients

In [ ]:
base_coeffs <- c(
  age             = 0.3,
  age_sq          = -0.002,
  house_ownership = 5,
  employed    = 10,
  electricity     = 8,
  wallsbrick     =3,
  roofmetal     = 4,
  floordirt     = -5,
  areaurban    =7
)

Simulate normal random noise and create per capita consumption

In [ ]:
noise=rnorm(n,sd=3) #alternative for normal noise
#noise=rexp(n, rate = 1/10) - 10
base_coeffs = matrix(base_coeffs,ncol=1)

#df$consumption_pc=exp(B_0 + as.matrix(df)%*%base_coeffs+noise) #alternative for normal noise
df$consumption_pc=as.matrix(df[,-10])%*%base_coeffs+noise

head(df)

Visualize the density plot

In [ ]:
library(ggplot2)

ggplot(df, aes(x = consumption_pc)) +
  geom_density(fill = "skyblue", color = "blue") +
  theme_minimal()

**2. Predicting consumption**

Adjust linear model for consumption_pc

In [ ]:
mod0=lm(consumption_pc ~ . , df)
summary(mod0)

Visualize residuals

In [ ]:
df$residuals = mod0$residuals

ggplot(df, aes(x = residuals)) +
  geom_density(fill = "skyblue", color = "blue") +
  theme_minimal()


How does the prediction look compared to the actual distribution?

In [ ]:
df$predicted <- predict(mod0)

ggplot(df, aes(x = consumption_pc)) +
  geom_density(aes(fill = "Actual"), alpha = 0.5) +
  geom_density(aes(x = predicted, fill = "Predicted"), alpha = 0.5) +
  scale_fill_manual(values = c("Actual" = "skyblue", "Predicted" = "lightcoral")) +
  theme_minimal() +
  labs(title = "Density Plot of Actual vs. Predicted Consumption per Capita",
       x = "Consumption per Capita",
       fill = "Distribution")

More realistic: dropping a **few** covariates

In [ ]:
mod1=lm(consumption_pc ~ age+age_sq+areaurban +electricity+house_ownership+roofmetal+hhsize, df)
summary(mod1)

Visualize residuals

In [ ]:
df$residuals = mod1$residuals

ggplot(df, aes(x = residuals)) +
  geom_density(fill = "skyblue", color = "blue") +
  theme_minimal()

And the prediction...

In [ ]:
df$predicted <- predict(mod1)

ggplot(df, aes(x = consumption_pc)) +
  geom_density(aes(fill = "Actual"), alpha = 0.5) +
  geom_density(aes(x = predicted, fill = "Predicted"), alpha = 0.5) +
  scale_fill_manual(values = c("Actual" = "skyblue", "Predicted" = "lightcoral")) +
  theme_minimal() +
  labs(title = "Density Plot of Actual vs. Predicted Consumption per Capita",
       x = "Consumption per Capita",
       fill = "Distribution")

Estimating poverty rate. Assume poverty line = 32.

In [ ]:
# Actual
cat("Actual poverty rate: ",mean(df$consumption_pc<32),"\n")

# Predicted
cat("Predicted poverty rate: ",mean(df$predicted<32))

Good enough. What if poverty line = 27

In [ ]:
# Actual
cat("Actual poverty rate: ",mean(df$consumption_pc<27),"\n")

# Predicted
cat("Predicted poverty rate: ",mean(df$predicted<27))

Uh-oh. Something went wrong. Let's explore.

In [ ]:
ggplot(df, aes(x = consumption_pc)) +
  stat_ecdf(aes(color = "Actual")) +
  stat_ecdf(aes(x = predicted, color = "Predicted")) +
  scale_color_manual(values = c("Actual" = "skyblue", "Predicted" = "lightcoral")) +
  theme_minimal() +
  labs(title = "ECDF Plot of Actual vs. Predicted Consumption per Capita",
       x = "Consumption per Capita",
       color = "Distribution")

Let's try to get a better hat using stepwise selection

In [ ]:
mod2=step(mod1)
summary(mod2)

In [ ]:
df$predicted <- predict(mod2)
# Actual
cat("Actual poverty rate: ",mean(df$consumption_pc<27),"\n")

# Predicted
cat("Predicted poverty rate: ",mean(df$predicted<27))


One more try. Let's lasso it.

In [ ]:
x <- model.matrix(consumption_pc ~ age+age_sq+areaurban +electricity+house_ownership+roofmetal+hhsize, df)[,-1]
y <- df$consumption_pc
lasso_model <- glmnet(x, y, alpha = 1)
df$predicted_lasso <- predict(lasso_model, newx = x, s = min(lasso_model$lambda)) # Use the minimum lambda

# Actual
cat("Actual poverty rate: ",mean(df$consumption_pc<27),"\n")

# Predicted Lasso
cat("Predicted lasso poverty rate: ",mean(df$predicted_lasso<27))

Not quite. How about the distribution

In [ ]:
ggplot(df, aes(x = consumption_pc)) +
  stat_ecdf(aes(color = "Actual")) +
  stat_ecdf(aes(x = predicted_lasso, color = "Predicted")) +
  scale_color_manual(values = c("Actual" = "skyblue", "Predicted" = "lightcoral")) +
  theme_minimal() +
  labs(title = "ECDF Plot of Actual vs. Predicted Consumption per Capita",
       x = "Consumption per Capita",
       color = "Distribution")